# Question 1: Stranded Assets Analysis - Carbon Pricing Impact

## Executive Summary
This analysis identifies which copper mining assets could become stranded under various carbon pricing scenarios ($50-$200/tCO₂). We examine:
- **Total exposure**: Absolute carbon costs by asset and company
- **Intensity exposure**: Carbon cost per unit of production
- **Geographic concentration**: Regional stranded asset risk
- **Portfolio recommendations**: Asset rebalancing strategies


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')

# Professional styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Color palette for scenarios
SCENARIO_COLORS = {
    '$50/tCO₂': '#2E86AB',
    '$100/tCO₂': '#A23B72',
    '$150/tCO₂': '#F18F01',
    '$200/tCO₂': '#C73E1D'
}

print('Successful imports')

Successful imports


## 1. Data Loading and Initial Assessment

In [21]:
df = pd.read_csv('copper_mining_cleaned.csv')

print(f"Dataset: {len(df):,} copper mining assets")
print(f"Countries: {df['iso3_country'].nunique()}")
print(f"Parent Companies: {df['parent_name'].nunique()} (with ownership data)")
print(f"Mine Status Distribution:")
print(df['mine_status'].value_counts())
print(f"Total Annual Emissions (2024): {df['annual_emissions_t_co2e'].sum():,.0f} tCO₂e")

Dataset: 914 copper mining assets
Countries: 55
Parent Companies: 256 (with ownership data)
Mine Status Distribution:
mine_status
Production    608
Closed        192
Suspended     114
Name: count, dtype: int64
Total Annual Emissions (2024): 95,204,706 tCO₂e


## 2. Carbon Cost Exposure Analysis

### 2.1 Asset-Level Rankings - Total Exposure

In [22]:
# Filter to active production mines for primary analysis
df_production = df[df['mine_status'] == 'Production'].copy()

# Calculate rankings for each scenario
scenarios = {
    '$50/tCO₂': 'carbon_cost_usd_50',
    '$100/tCO₂': 'carbon_cost_usd_100',
    '$150/tCO₂': 'carbon_cost_usd_150',
    '$200/tCO₂': 'carbon_cost_usd_200'
}

# Top 20 most exposed assets (at $100/tCO₂)
top_exposed = df_production.nlargest(20, 'carbon_cost_usd_100')[[
    'source_name', 'iso3_country', 'parent_name', 'annual_emissions_t_co2e',
    'carbon_cost_usd_50', 'carbon_cost_usd_100', 'carbon_cost_usd_150', 
    'carbon_cost_usd_200', 'carbon_intensity', 'emissions_confidence'
]].copy()

top_exposed.columns = ['Mine', 'Country', 'Parent Company', 'Annual Emissions (tCO₂)',
                       '$50/t', '$100/t', '$150/t', '$200/t', 'Intensity', 'Confidence']

print("Top 20 Assets by Total Carbon Cost Exposure (Production Mines)")

display(top_exposed.style.format({
    'Annual Emissions (tCO₂)': '{:,.0f}',
    '$50/t': '${:,.0f}',
    '$100/t': '${:,.0f}',
    '$150/t': '${:,.0f}',
    '$200/t': '${:,.0f}',
    'Intensity': '{:.4f}'
}).background_gradient(subset=['$100/t'], cmap='Reds'))

Top 20 Assets by Total Carbon Cost Exposure (Production Mines)


,Mine,Country,Parent Company,Annual Emissions (tCO₂),$50/t,$100/t,$150/t,$200/t,Intensity,Confidence
557,Sarcheshmeh Complex,IRN,Government of Iran,"2,601,350","$130,067,500","$260,135,000","$390,202,500","$520,270,000",0.0240,very low
668,Antapaccay Mine,PER,Qatar Investment Authority,"1,863,345","$93,167,250","$186,334,500","$279,501,750","$372,669,000",0.0455,medium
378,Dikuluwe-Mashamba West Mine,COD,SinoCongolaise des Mines SA,"1,791,061","$89,553,050","$179,106,100","$268,659,150","$358,212,200",0.0414,low
549,Grasberg Block Cave Mine,IDN,FreePort-McMoran Inc,"1,592,699","$79,634,951","$159,269,901","$238,904,852","$318,539,802",0.0372,medium
650,Letpadaung Mine,MMR,Wanbao Mining Co,"1,348,314","$67,415,700","$134,831,400","$202,247,100","$269,662,800",0.0240,very low
524,Tenke Fungurume Mine,COD,CMOC Group Ltd,"1,340,404","$67,020,200","$134,040,400","$201,060,600","$268,080,800",0.0944,high
558,Sungun Mine,IRN,Government of Iran,"1,192,794","$59,639,700","$119,279,400","$178,919,100","$238,558,800",0.0240,very low
554,Jian Mine,IRN,nan,"1,156,495","$57,824,762","$115,649,524","$173,474,285","$231,299,047",0.0240,very low
255,Minera Valle Central Mine,CHL,Amerigo Resources Ltd,"1,067,961","$53,398,050","$106,796,100","$160,194,150","$213,592,200",0.0193,medium
888,Kansanshi Mine,ZMB,First Quantum Minerals Ltd,"1,058,410","$52,920,500","$105,841,000","$158,761,500","$211,682,000",0.0454,high


### 2.2 Asset-Level Rankings - Intensity Exposure

In [23]:
# Filter mines with calculable intensity
df_intensity = df_production[df_production['include_in_intensity'] == True].copy()

# Calculate cost per tonne of ore produced
for scenario_name, col in scenarios.items():
    df_intensity[f'cost_per_t_ore_{scenario_name}'] = (
        df_intensity[col] / df_intensity['annual_production_t_ore']
    )

# Top 20 by intensity exposure at $100/tCO₂
top_intensity = df_intensity.nlargest(20, 'cost_per_t_ore_$100/tCO₂')[[
    'source_name', 'iso3_country', 'parent_name', 'carbon_intensity',
    'cost_per_t_ore_$50/tCO₂', 'cost_per_t_ore_$100/tCO₂', 
    'cost_per_t_ore_$150/tCO₂', 'cost_per_t_ore_$200/tCO₂',
    'annual_production_t_ore', 'emissions_confidence'
]].copy()

top_intensity.columns = ['Mine', 'Country', 'Parent Company', 'Carbon Intensity',
                        'Cost/t Ore ($50)', 'Cost/t Ore ($100)', 'Cost/t Ore ($150)', 
                        'Cost/t Ore ($200)', 'Production (t)', 'Confidence']

print('Top 20 Assests by carbon exposure' )

display(top_intensity.style.format({
    'Carbon Intensity': '{:.4f}',
    'Cost/t Ore ($50)': '${:.2f}',
    'Cost/t Ore ($100)': '${:.2f}',
    'Cost/t Ore ($150)': '${:.2f}',
    'Cost/t Ore ($200)': '${:.2f}',
    'Production (t)': '{:,.0f}'
}).background_gradient(subset=['Cost/t Ore ($100)'], cmap='Oranges'))

Top 20 Assests by carbon exposure


,Mine,Country,Parent Company,Carbon Intensity,Cost/t Ore ($50),Cost/t Ore ($100),Cost/t Ore ($150),Cost/t Ore ($200),Production (t),Confidence
212,El Salvador Mine,CHL,Codelco Corp,0.3005,$15.03,$30.05,$45.08,$60.10,"720,242",medium
607,Sepon Mine,LAO,Chifeng Jilong Gold Mining Co Ltd,0.2764,$13.82,$27.64,$41.46,$55.28,"400,000",high
805,Las Cruces Mine,ESP,Cobre Las Cruces SA,0.1403,$7.02,$14.03,$21.05,$28.06,"35,636",high
66,Deflector Operation,AUS,Silver Lake Resources Ltd,0.0989,$4.95,$9.89,$14.84,$19.78,"1,065,446",medium
524,Tenke Fungurume Mine,COD,CMOC Group Ltd,0.0944,$4.72,$9.44,$14.16,$18.88,"14,199,200",high
27,Eloise Mine,AUS,AIC Mines Ltd,0.0809,$4.05,$8.09,$12.14,$16.18,"574,400",high
91,Palito Mine,BRA,Serabi Gold PLC,0.0774,$3.87,$7.74,$11.61,$15.48,"177,870",high
50,Nova Mine,AUS,IGO Ltd,0.0740,$3.70,$7.40,$11.10,$14.80,"1,500,101",high
662,Kainantu Mine,PNG,K92 Holdings International Ltd,0.0675,$3.37,$6.75,$10.12,$13.50,"506,318",high
545,Batu Hijau Mine,IDN,PT Amman Mineral Internasional Tbk,0.0593,$2.96,$5.93,$8.89,$11.86,"11,493,361",medium


## 3. Company-Level Portfolio Aggregation

In [24]:
# Aggregate by parent company (production mines only)
company_exposure = df_production[df_production['has_ownership'] == True].groupby(
    ['parent_name', 'parent_headquarter_country']
).agg({
    'source_id': 'count',
    'annual_emissions_t_co2e': 'sum',
    'annual_production_t_ore': 'sum',
    'carbon_cost_usd_50': 'sum',
    'carbon_cost_usd_100': 'sum',
    'carbon_cost_usd_150': 'sum',
    'carbon_cost_usd_200': 'sum'
}).reset_index()

company_exposure.columns = ['Company', 'HQ Country', 'Mines', 'Total Emissions (tCO₂)',
                           'Production (t)', '$50/t', '$100/t', '$150/t', '$200/t']

# Calculate portfolio carbon intensity
company_exposure['Portfolio Intensity'] = (
    company_exposure['Total Emissions (tCO₂)'] / company_exposure['Production (t)']
)

# Top 25 companies by exposure
top_companies = company_exposure.nlargest(25, '$100/t').copy()

print('Top 25 companies by carbon cost exposure' )

display(top_companies.style.format({
    'Mines': '{:,.0f}',
    'Total Emissions (tCO₂)': '{:,.0f}',
    'Production (t)': '{:,.0f}',
    '$50/t': '${:,.0f}',
    '$100/t': '${:,.0f}',
    '$150/t': '${:,.0f}',
    '$200/t': '${:,.0f}',
    'Portfolio Intensity': '{:.4f}'
}).background_gradient(subset=['$100/t', 'Portfolio Intensity'], cmap='RdYlGn_r'))

Top 25 companies by carbon cost exposure


,Company,HQ Country,Mines,Total Emissions (tCO₂),Production (t),$50/t,$100/t,$150/t,$200/t,Portfolio Intensity
12,FreePort-McMoran Inc,USA,11,"4,858,750","468,335,535","$242,937,501","$485,875,002","$728,812,503","$971,750,004",0.0104
16,Government of Iran,IRN,3,"4,844,845","201,868,552","$242,242,250","$484,484,500","$726,726,750","$968,969,000",0.0240
29,Qatar Investment Authority,QAT,11,"2,318,213","52,153,138","$115,910,650","$231,821,300","$347,731,950","$463,642,600",0.0445
24,Kazakhmys Holding LLP,KAZ,4,"2,262,216","403,967,130","$113,110,802","$226,221,604","$339,332,406","$452,443,208",0.0056
31,The Vanguard Group Inc,USA,14,"2,091,091","461,844,775","$104,554,550","$209,109,100","$313,663,650","$418,218,200",0.0045
32,Zijin Mining Group Co Ltd,CHN,10,"1,641,364","103,724,303","$82,068,221","$164,136,443","$246,204,664","$328,272,886",0.0158
9,Eurasian Resources Group SARL,LUX,5,"1,400,075","152,311,046","$70,003,765","$140,007,530","$210,011,294","$280,015,059",0.0092
11,First Quantum Minerals Ltd,CAN,5,"1,384,564","143,200,001","$69,228,200","$138,456,400","$207,684,600","$276,912,800",0.0097
14,Glencore PLC,CHE,2,"1,345,730","29,576,475","$67,286,500","$134,573,000","$201,859,500","$269,146,000",0.0455
18,Government of Poland,POL,6,"1,203,982","94,061,202","$60,199,100","$120,398,200","$180,597,300","$240,796,400",0.0128


## 4. Visualization Dashboard

### 4.1 Bar Chart: Top 15 Most Exposed Assets

In [25]:
# Prepare data for visualization
top15_assets = df_production.nlargest(15, 'carbon_cost_usd_100')[[
    'source_name', 'iso3_country', 'carbon_cost_usd_50', 'carbon_cost_usd_100',
    'carbon_cost_usd_150', 'carbon_cost_usd_200'
]].copy()

# Reshape for grouped bar chart
top15_melted = top15_assets.melt(
    id_vars=['source_name', 'iso3_country'],
    value_vars=['carbon_cost_usd_50', 'carbon_cost_usd_100', 'carbon_cost_usd_150', 'carbon_cost_usd_200'],
    var_name='Scenario',
    value_name='Carbon Cost (USD)'
)

# Map scenario names
scenario_map = {
    'carbon_cost_usd_50': '$50/tCO₂',
    'carbon_cost_usd_100': '$100/tCO₂',
    'carbon_cost_usd_150': '$150/tCO₂',
    'carbon_cost_usd_200': '$200/tCO₂'
}
top15_melted['Scenario'] = top15_melted['Scenario'].map(scenario_map)

# Create interactive bar chart
fig1 = px.bar(
    top15_melted,
    x='source_name',
    y='Carbon Cost (USD)',
    color='Scenario',
    barmode='group',
    title='Top 15 Assets by Carbon Cost Exposure (Multiple Scenarios)',
    labels={'source_name': 'Mine', 'Carbon Cost (USD)': 'Annual Carbon Cost (USD)'},
    color_discrete_map=SCENARIO_COLORS,
    height=600
)

fig1.update_layout(
    xaxis_tickangle=-45,
    font=dict(size=11),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig1.show()

### 4.2 Scatter Plot: Carbon Intensity vs. Total Emissions

In [ ]:
# Create scatter plot with production mines
scatter_data = df_intensity.copy()
scatter_data['size_marker'] = np.sqrt(
    scatter_data['annual_production_t_ore']) # Scale for visibility

fig2 = px.scatter(
    scatter_data,
    x='carbon_intensity',
    y='carbon_cost_usd_100',
    size='size_marker',
    color='iso3_country',
    hover_data=['source_name', 'parent_name', 'annual_emissions_t_co2e', 'annual_production_t_ore'],
    title='Carbon Intensity vs. Total Exposure ($100/tCO₂ Scenario)',
    labels={
        'carbon_intensity': 'Carbon Intensity (tCO₂/t ore)',
        'carbon_cost_usd_100': 'Annual Carbon Cost at $100/t (USD)',
        'iso3_country': 'Country'
    },
    height=600
)

fig2.update_layout(
    showlegend=False,  # Too many countries
    font=dict(size=11)
)

fig2.show()


Key Insight: Assets in the top-right quadrant (high intensity + high cost) face the greatest stranded asset risk.

### 4.3 Geographic Map: Regional Stranded Asset Risk

In [ ]:
# Prepare geographic data
map_data = df_production[df_production['carbon_cost_usd_100'] > 0].copy()

fig3 = px.scatter_geo(
    map_data,
    lat='lat',
    lon='lon',
    size='carbon_cost_usd_100',
    color='carbon_intensity',
    hover_name='source_name',
    hover_data={
        'iso3_country': True,
        'parent_name': True,
        'carbon_cost_usd_100': ':$,.0f',
        'annual_emissions_t_co2e': ':,.0f',
        'carbon_intensity': ':.4f',
        'lat': False,
        'lon': False
    },
    title='Global Distribution of Stranded Asset Risk ($100/tCO₂ Scenario)',
    size_max=30,
    height=600
)

fig3.update_layout(
    geo=dict(
        projection_type='natural earth',
        showland=True
    )
)

fig3.show()


Bubble size = Total carbon cost | Color intensity = Carbon intensity per tonne ore

### 4.4 Country-Level Aggregation

In [28]:
# Aggregate by country
country_exposure = df_production.groupby('iso3_country').agg({
    'source_id': 'count',
    'annual_emissions_t_co2e': 'sum',
    'carbon_cost_usd_50': 'sum',
    'carbon_cost_usd_100': 'sum',
    'carbon_cost_usd_150': 'sum',
    'carbon_cost_usd_200': 'sum'
}).reset_index()

country_exposure.columns = ['Country', 'Mines', 'Emissions (tCO₂)',
                           '$50/t', '$100/t', '$150/t', '$200/t']

top_countries = country_exposure.nlargest(15, '$100/t')

fig4 = go.Figure(data=[
    go.Bar(name='$50/tCO₂', x=top_countries['Country'], y=top_countries['$50/t'], marker_color=SCENARIO_COLORS['$50/tCO₂']),
    go.Bar(name='$100/tCO₂', x=top_countries['Country'], y=top_countries['$100/t'], marker_color=SCENARIO_COLORS['$100/tCO₂']),
    go.Bar(name='$150/tCO₂', x=top_countries['Country'], y=top_countries['$150/t'], marker_color=SCENARIO_COLORS['$150/tCO₂']),
    go.Bar(name='$200/tCO₂', x=top_countries['Country'], y=top_countries['$200/t'], marker_color=SCENARIO_COLORS['$200/tCO₂'])
])

fig4.update_layout(
    title='Top 15 Countries by Carbon Cost Exposure (Production Mines)',
    xaxis_title='Country',
    yaxis_title='Annual Carbon Cost (USD)',
    barmode='group',
    height=600,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig4.show()

## 5. Stranded Asset Identification

### 5.1 Define Risk Categories

In [29]:
# Calculate percentiles for risk classification
p75_cost = df_production['carbon_cost_usd_100'].quantile(0.75)
p75_intensity = df_intensity['carbon_intensity'].quantile(0.75)

# Create risk categories
def classify_risk(row):
    if row['mine_status'] != 'Production':
        return 'Already Stranded'
    
    high_cost = row['carbon_cost_usd_100'] > p75_cost
    high_intensity = row.get('carbon_intensity', 0) > p75_intensity if pd.notna(row.get('carbon_intensity')) else False
    
    if high_cost and high_intensity:
        return 'Critical Risk'
    elif high_cost or high_intensity:
        return 'High Risk'
    elif row['carbon_cost_usd_100'] > 0:
        return 'Moderate Risk'
    else:
        return 'Low Risk'

df['risk_category'] = df.apply(classify_risk, axis=1)

# Risk distribution
risk_dist = df['risk_category'].value_counts()
print("Stranded Asset Risk Distribution")
print(risk_dist)
print(f"Percentage at Critical/High Risk: {(risk_dist.get('Critical Risk', 0) + risk_dist.get('High Risk', 0)) / len(df) * 100:.1f}%")

Stranded Asset Risk Distribution
risk_category
Moderate Risk       342
Already Stranded    306
High Risk           238
Critical Risk        21
Low Risk              7
Name: count, dtype: int64
Percentage at Critical/High Risk: 28.3%


### 5.2 Critical Risk Assets - Detailed Analysis

In [30]:
# Identify critical risk assets
critical_assets = df[df['risk_category'] == 'Critical Risk'][[
    'source_name', 'iso3_country', 'parent_name', 'mine_type',
    'annual_emissions_t_co2e', 'carbon_intensity', 'carbon_cost_usd_100',
    'carbon_cost_usd_200', 'capacity_factor', 'emissions_confidence'
]].copy()

critical_assets.columns = ['Mine', 'Country', 'Parent', 'Type', 'Emissions', 
                          'Intensity', 'Cost@$100', 'Cost@$200', 'Capacity', 'Confidence']

print(f"Critical Risk Assets (n={len(critical_assets)})")
print("High carbon cost AND high intensity - Most vulnerable to carbon pricing")

display(critical_assets.sort_values('Cost@$100', ascending=False).head(30).style.format({
    'Emissions': '{:,.0f}',
    'Intensity': '{:.4f}',
    'Cost@$100': '${:,.0f}',
    'Cost@$200': '${:,.0f}',
    'Capacity': '{:.2%}'
}).background_gradient(subset=['Intensity', 'Cost@$100'], cmap='Reds'))

Critical Risk Assets (n=21)
High carbon cost AND high intensity - Most vulnerable to carbon pricing


,Mine,Country,Parent,Type,Emissions,Intensity,Cost@$100,Cost@$200,Capacity,Confidence
668,Antapaccay Mine,PER,Qatar Investment Authority,Open Pit,"1,863,345",0.0455,"$186,334,500","$372,669,000",8.92%,medium
378,Dikuluwe-Mashamba West Mine,COD,SinoCongolaise des Mines SA,Open Pit,"1,791,061",0.0414,"$179,106,100","$358,212,200",1.51%,low
549,Grasberg Block Cave Mine,IDN,FreePort-McMoran Inc,Underground,"1,592,699",0.0372,"$159,269,901","$318,539,802",5.51%,medium
524,Tenke Fungurume Mine,COD,CMOC Group Ltd,Open Pit,"1,340,404",0.0944,"$134,040,400","$268,080,800",5.12%,high
888,Kansanshi Mine,ZMB,First Quantum Minerals Ltd,Open Pit,"1,058,410",0.0454,"$105,841,000","$211,682,000",2.63%,high
240,Lomas Bayas Mine,CHL,Glencore PLC,Open Pit,"1,038,655",0.0455,"$103,865,500","$207,731,000",8.04%,medium
547,Deep Mill Level Zone Mine,IDN,FreePort-McMoran Inc,Underground,"1,030,570",0.0372,"$103,057,001","$206,114,001",8.32%,medium
545,Batu Hijau Mine,IDN,PT Amman Mineral Internasional Tbk,Open Pit,"681,556",0.0593,"$68,155,600","$136,311,201",1.43%,medium
368,Pueblo Viejo Mine,DOM,Barrick Gold Corp,Open Pit,"479,331",0.0369,"$47,933,100","$95,866,200",4.48%,high
518,Sico Mine,COD,China RAILWAY Group Ltd,Open Pit,"396,978",0.0414,"$39,697,800","$79,395,600",1.51%,low


## 6. Scenario Sensitivity Analysis

In [31]:
# Calculate total global exposure by scenario
scenario_totals = {
    '$50/tCO₂': df_production['carbon_cost_usd_50'].sum(),
    '$100/tCO₂': df_production['carbon_cost_usd_100'].sum(),
    '$150/tCO₂': df_production['carbon_cost_usd_150'].sum(),
    '$200/tCO₂': df_production['carbon_cost_usd_200'].sum()
}

# Scenario comparison
fig5 = go.Figure()

fig5.add_trace(go.Bar(
    x=list(scenario_totals.keys()),
    y=list(scenario_totals.values()),
    marker_color=[SCENARIO_COLORS[s] for s in scenario_totals.keys()],
    text=[f'${v/1e9:.2f}B' for v in scenario_totals.values()],
    textposition='outside'
))

fig5.update_layout(
    title='Global Carbon Cost Exposure - Scenario Comparison (Production Mines Only)',
    xaxis_title='Carbon Price Scenario',
    yaxis_title='Total Annual Carbon Cost (USD)',
    height=500
)

fig5.show()

print("Scenario Sensitivity Summary")

for scenario, cost in scenario_totals.items():
    print(f"{scenario:15} : ${cost:>15,.0f}  (${cost/1e9:.2f} billion)")

print(f"\nCost increase from $50 to $200: ${(scenario_totals['$200/tCO₂'] - scenario_totals['$50/tCO₂'])/1e9:.2f}B (+{(scenario_totals['$200/tCO₂']/scenario_totals['$50/tCO₂'] - 1)*100:.0f}%)")

Scenario Sensitivity Summary
$50/tCO₂        : $  4,760,235,286  ($4.76 billion)
$100/tCO₂       : $  9,520,470,572  ($9.52 billion)
$150/tCO₂       : $ 14,280,705,858  ($14.28 billion)
$200/tCO₂       : $ 19,040,941,144  ($19.04 billion)

Cost increase from $50 to $200: $14.28B (+300%)


## 7. Portfolio Rebalancing Recommendations

### 7.1 Identify Divestment Candidates

In [32]:
# Divestment criteria: Critical risk OR (High risk + low capacity factor)
divestment_candidates = df[
    (df['risk_category'] == 'Critical Risk') |
    ((df['risk_category'] == 'High Risk') & (df['capacity_factor'] < 0.3))
].copy()

divest_summary = divestment_candidates.groupby('parent_name').agg({
    'source_id': 'count',
    'carbon_cost_usd_100': 'sum',
    'annual_emissions_t_co2e': 'sum'
}).reset_index()

divest_summary.columns = ['Company', 'Assets at Risk', 'Exposure@$100/t', 'Emissions']
divest_summary = divest_summary.sort_values('Exposure@$100/t', ascending=False).head(20)


print("Top 20 Companies - Potential Divestment Portfolio")
print("Critical risk assets + underperforming high-risk assets (capacity factor < 30%)")

display(divest_summary.style.format({
    'Assets at Risk': '{:.0f}',
    'Exposure@$100/t': '${:,.0f}',
    'Emissions': '{:,.0f}'
}).background_gradient(subset=['Exposure@$100/t'], cmap='Reds'))

Top 20 Companies - Potential Divestment Portfolio
Critical risk assets + underperforming high-risk assets (capacity factor < 30%)


,Company,Assets at Risk,Exposure@$100/t,Emissions
26,Government of Iran,3,"$484,484,500","4,844,845"
23,FreePort-McMoran Inc,6,"$438,491,502","4,384,915"
52,Qatar Investment Authority,5,"$211,210,300","2,112,103"
63,Wanbao Mining Co,3,"$208,164,500","2,081,645"
57,SinoCongolaise des Mines SA,1,"$179,106,100","1,791,061"
67,Zijin Mining Group Co Ltd,5,"$143,079,743","1,430,797"
19,Codelco Corp,4,"$140,686,500","1,406,865"
59,Southern Copper Corp,3,"$140,406,700","1,404,067"
22,First Quantum Minerals Ltd,2,"$136,368,900","1,363,689"
24,Glencore PLC,2,"$134,573,000","1,345,730"


### 7.2 Identify Low-Risk Investment Opportunities

In [33]:
# Low carbon intensity + high capacity factor = resilient assets
low_risk_assets = df_intensity[
    (df_intensity['carbon_intensity'] < df_intensity['carbon_intensity'].quantile(0.25)) &
    (df_intensity['capacity_factor'] > 0.5) &
    (df_intensity['annual_production_t_ore'] > df_intensity['annual_production_t_ore'].median())
][[
    'source_name', 'iso3_country', 'parent_name', 'carbon_intensity',
    'capacity_factor', 'annual_production_t_ore', 'carbon_cost_usd_100',
    'emissions_confidence'
]].copy()

low_risk_assets.columns = ['Mine', 'Country', 'Parent', 'Intensity', 
                          'Capacity', 'Production', 'Cost@$100', 'Confidence']

print(f"Low-Risk Investment Opportunities (n={len(low_risk_assets)})")
print("Low carbon intensity + high capacity utilization + above-median production")

display(low_risk_assets.sort_values('Intensity').head(20).style.format({
    'Intensity': '{:.4f}',
    'Capacity': '{:.2%}',
    'Production': '{:,.0f}',
    'Cost@$100': '${:,.0f}'
}).background_gradient(subset=['Intensity'], cmap='Greens_r'))

Low-Risk Investment Opportunities (n=28)
Low carbon intensity + high capacity utilization + above-median production


,Mine,Country,Parent,Intensity,Capacity,Production,Cost@$100,Confidence
576,Kounrad Mine,KAZ,Central Asia Metals PLC,0.0001,68.01%,"193,666,059","$1,936,700",medium
667,Antamina Mine,PER,The Vanguard Group Inc,0.0025,100.00%,"245,682,000","$61,420,500",medium
791,Kroondal Mine,ZAF,Public Investment Corporation SOC Ltd,0.0050,59.87%,"5,448,000","$2,724,000",high
603,Zhomart Mine,KAZ,nan,0.0056,68.01%,"132,094,467","$73,972,901",very low
602,Zhilandinsky Mine,KAZ,nan,0.0056,68.01%,"132,094,467","$73,972,901",very low
601,Zhezkazgan West Mine,KAZ,nan,0.0056,68.01%,"132,094,467","$73,972,901",very low
599,Zhezkazgan East Mine,KAZ,nan,0.0056,68.01%,"132,094,467","$73,972,901",very low
598,Zhairem Zapadny Mine,KAZ,Eurasian Resources Group SARL,0.0056,68.01%,"132,094,467","$73,972,901",medium
593,Stepnoy Mine,KAZ,nan,0.0056,68.01%,"132,094,467","$73,972,901",very low
591,Shatyrkul Mine,KAZ,Kazakhmys Holding LLP,0.0056,68.01%,"132,094,467","$73,972,901",very low


## 8. Key Findings and Strategic Recommendations

In [34]:
total_production_mines = len(df_production)
critical_risk_count = len(df[df['risk_category'] == 'Critical Risk'])
high_risk_count = len(df[df['risk_category'] == 'High Risk'])
total_exposure_100 = df_production['carbon_cost_usd_100'].sum()
median_intensity = df_intensity['carbon_intensity'].median()
top_10_pct_exposure = df_production.nlargest(int(len(df_production)*0.1), 'carbon_cost_usd_100')['carbon_cost_usd_100'].sum() / total_exposure_100
top_3_country_pct = country_exposure.nlargest(3, '$100/t')['$100/t'].sum()/country_exposure['$100/t'].sum()*100
cost_per_10_increase = (scenario_totals['$100/tCO₂'] - scenario_totals['$50/tCO₂'])/5/1e9
critical_cost_escalation = (critical_assets['Cost@$200'].sum() / critical_assets['Cost@$100'].sum() - 1)*100
medium_high_confidence_pct = (df_production['emissions_confidence'].isin(['high', 'medium']).sum() / len(df_production) * 100)
ownership_coverage_pct = (df_production['has_ownership'].sum() / len(df_production) * 100)

### Executive Summary: Stranded Assets under Carbon Pricing

---

#### 1. PORTFOLIO RISK ASSESSMENT

- **Production mines analyzed**: 608
- **Critical risk assets**: 21 (2.3% of portfolio)
- **High risk assets**: 238 (26.0% of portfolio)
- **Combined at-risk portfolio**: 259 assets (28.3%)

#### 2. FINANCIAL EXPOSURE (Annual, Production Mines)

| Scenario | Annual Cost | Status |
|----------|-------------|--------|
| **$50/tCO₂** | $4.76 billion | Low ambition |
| **$100/tCO₂** | $9.52 billion | **Base case** |
| **$150/tCO₂** | $14.28 billion | High ambition |
| **$200/tCO₂** | $19.04 billion | Net-zero transition |

**Scenario range**: $14.28B swing between low and high scenarios

#### 3. CONCENTRATION RISK

- **Top 10% of assets** account for **51.6%** of total exposure
- **Median carbon intensity**: 0.0150 tCO₂/t ore
- **Geographic concentration**: Top 3 countries hold **45.7%** of exposure

#### 4. STRATEGIC RECOMMENDATIONS

##### A. IMMEDIATE ACTIONS (0-12 months)
- Conduct detailed asset-level carbon audits for **21 critical risk mines**
- Initiate divestment analysis for underperforming high-risk assets
- Develop decarbonization roadmaps for strategically important high-emission assets
- Enhance carbon cost disclosure in financial reporting (scenario analysis)

##### B. PORTFOLIO REBALANCING (1-3 years)
- Prioritize **28 low-intensity, high-capacity assets** for expansion
- Consider partial divestment of critical risk assets with low strategic value
- Shift capital allocation toward low-carbon production regions
- Implement carbon performance metrics in M&A screening

##### C. LONG-TERM RESILIENCE (3-10 years)
- Target portfolio average carbon intensity reduction of **30-50%**
- Invest in renewable energy for high-emission, high-value assets
- Develop partnerships for carbon capture/offset programs
- Build optionality through renewable copper exposure (recycling, low-carbon sources)

#### 5. SENSITIVITY TO CARBON PRICE ASSUMPTIONS

- Every **$10/tCO₂ increase** = **$0.95B** additional annual cost
- At $200/tCO₂, carbon costs could exceed **100%** of current $100 scenario
- Critical risk assets face **100%** cost escalation from $100→$200 pricing

#### 6. DATA QUALITY CONSIDERATIONS

- **41.0%** of production mines have medium-high confidence emissions data
- **66.8%** of production mines have parent company ownership mapping
- **Recommendation**: Prioritize data quality improvements for top 20% exposed assets

------

## 9. Export Results for Further Analysis

In [35]:
# Export key datasets
critical_assets.to_csv('critical_risk_assets.csv', index=False)
top_companies.to_csv('company_carbon_exposure.csv', index=False)
divest_summary.to_csv('divestment_candidates.csv', index=False)
low_risk_assets.to_csv('low_risk_opportunities.csv', index=False)

# Export full dataset with risk categories
df.to_csv('copper_mining_with_risk_categories.csv', index=False)

print(" Exported files:")
print("  - critical_risk_assets.csv")
print("  - company_carbon_exposure.csv")
print("  - divestment_candidates.csv")
print("  - low_risk_opportunities.csv")
print("  - copper_mining_with_risk_categories.csv")

 Exported files:
  - critical_risk_assets.csv
  - company_carbon_exposure.csv
  - divestment_candidates.csv
  - low_risk_opportunities.csv
  - copper_mining_with_risk_categories.csv


## Methodology Notes

### Risk Classification Criteria
- **Critical Risk**: Production mines with BOTH high total carbon cost (>75th percentile) AND high intensity (>75th percentile)
- **High Risk**: Production mines with EITHER high cost OR high intensity
- **Moderate Risk**: Production mines with positive carbon costs but below high-risk thresholds
- **Low Risk**: Minimal carbon costs
- **Already Stranded**: Closed or suspended mines

### Carbon Price Scenarios
Based on IPCC pathway modeling and current carbon pricing mechanisms:
- **$50/tCO₂**: Low ambition scenario (current voluntary market)
- **$100/tCO₂**: Base case (EU ETS 2023-24 levels, Paris Agreement alignment)
- **$150/tCO₂**: High ambition (1.5°C pathway)
- **$200/tCO₂**: Very high ambition (early net-zero transition)

### Data Sources
- **Emissions**: Climate Trace v5.2.0 (2024 annual aggregates from monthly data)
- **Ownership**: Climate Trace ownership mapping (58% coverage)
- **Production**: Climate Trace activity estimates with confidence flags

### Limitations
1. Analysis assumes static production levels (2024 baseline)
2. Does not account for potential mitigation investments (renewable energy, efficiency)
3. Carbon prices assumed uniform globally (actual prices vary by jurisdiction)
4. Ownership data incomplete for 42% of assets
5. Does not include Scope 3 emissions (downstream processing, transport)
